<a href="https://colab.research.google.com/github/MiscellaneousStuff/silent_speech_colab/blob/main/(Improved_%26_Finalised)_An_Improved_Model_for_Voicing_Silent_Speech_Implementation_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An Improved Model for Voicing Silent Speech - GPU/TPU Implementation

This notebook contains code to perform one of the following three operations using the model, EEG data and phoneme alignment data:
1. Train WaveNet on the full voice data
2. Train an EMG to Speech Feature Transduction Model (requires a pre-trained model for testing)
3. Evaluate ASR on the Trained Model

In [ ]:
ACCELERATOR = "gpu"
OPERATION   = "transduction_model" # "wavenet_model" or "transduction_model" or "evaluate"
OUTPUT_DIR  = "gdrive"        # "gdrive" or "local"

## Install Pre-Requisite Libraries

### Install Miniconda


In [ ]:
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6.7-hc745277_3 ...
installing: xz-5.2.4-h14c3975_4 ...
installing: yaml-0.1.7-had09818_2 ...
installing: zlib-1.2.11-ha838bed_2 ...
installing: libedit-3.1.20170329-h6b74fdf_2 ...
installing: readline-7.0-ha6073c6_4 ...
installing: sqlite-3.23.1-he433501_0 ...
installing: asn1crypto-0.24.0-py36_0 ...
installing: certifi-2018.4.16-py36_0 ...
installing: chardet-3.0.4-py36h0f667ec_1 ...
installing: idna-2.6-py36h82fb2a8_1 ...
installing: pycosat-0.6.3-py36h0a5515d_0 ...
installing: pycparser-2.18-py36hf9f622e_1 ...
installing: pysocks-1.6.8-py36_0 ...
installing: ruamel_yaml-0.15.37-py36h14c3975_2 ...
installing: six-1.11

--2021-07-19 00:10:07--  https://repo.continuum.io/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.200.79, 104.18.201.79, 2606:4700::6812:c84f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.200.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh [following]
--2021-07-19 00:10:07--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 45.6M 1s
    50K .......... .......... .......... .......... ..........  0%

#### Install Python 3.6 on Miniconda

In [ ]:
%%bash
conda install --channel defaults conda python=3.6 --yes
conda update --channel defaults --all --yes

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - conda
    - python=3.6


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libgcc-ng-9.1.0            |       hdf63c60_0         8.1 MB
    python-3.6.13              |       h12debd9_1        32.5 MB
    conda-package-handling-1.7.3|   py36h27cfd23_1         946 KB
    pycparser-2.20             |             py_2          94 KB
    zlib-1.2.11                |       h7b6447c_3         120 KB
    conda-4.10.3               |   py36h06a4308_0         3.1 MB
    pycosat-0.6.3              |   py36h27cfd23_0         107 KB
    libffi-3.3                 |       he6710b0_2          54 KB
    urllib3-1.26.6             |     pyhd3eb1b0_1         106 KB
    sqlite-3.36.0              |       hc218d9a_0         1.4 MB
    setuptools-52.0.0          |   py36h06a4308_0  

libgcc-ng-9.1.0      |  8.1 MB | ########## | 100% 
python-3.6.13        | 32.5 MB | ########## | 100% 
conda-package-handli |  946 KB | ########## | 100% 
pycparser-2.20       |   94 KB | ########## | 100% 
zlib-1.2.11          |  120 KB | ########## | 100% 
conda-4.10.3         |  3.1 MB | ########## | 100% 
pycosat-0.6.3        |  107 KB | ########## | 100% 
libffi-3.3           |   54 KB | ########## | 100% 
urllib3-1.26.6       |  106 KB | ########## | 100% 
sqlite-3.36.0        |  1.4 MB | ########## | 100% 
setuptools-52.0.0    |  933 KB | ########## | 100% 
xz-5.2.5             |  438 KB | ########## | 100% 
six-1.16.0           |   18 KB | ########## | 100% 
readline-8.1         |  464 KB | ########## | 100% 
pyopenssl-20.0.1     |   48 KB | ########## | 100% 
tk-8.6.10            |  3.2 MB | ########## | 100% 
chardet-4.0.0        |  213 KB | ########## | 100% 
ruamel_yaml-0.15.100 |  268 KB | ########## | 100% 
tqdm-4.61.2          |   80 KB | ########## | 100% 
pip-21.1.3  

#### Append Miniconda Python 3.6 to Path

In [ ]:
import sys
_ = (sys.path
        .append("/usr/local/lib/python3.6/site-packages"))

### Install Conda Packages

In [ ]:
!conda install --channel conda-forge featuretools --yes

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / failed with initial frozen solve. Retrying with flexible solve.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | 

In [ ]:
!conda install cudatoolkit=10.1 -y
!conda install pytorch -c pytorch -y
!conda install libsndfile=1.0.28 -c conda-forge -y
!conda install ipykernel -y

Solving environment: - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cudatoolkit=10.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.7.5   |       h06a4308_1         113 KB
    certifi-2021.5.30          |   py36h06a4308_0         139 KB
    conda-4.10.3               |   py36h06a4308_0         2.9 MB
    cudatoolkit-10.1.243       |       h6bb024c_0       347.4 MB
    openssl-1.1.1k             |       h27cfd23_0         2.5 MB
    ------------------------------------------------------------
                                           Total:       353.1 MB

The following NEW packages will be INSTALLED:

  cudatoolkit        pkgs/main/linux-64::cudatoolkit-10.1.243-h6bb024c_0

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certific

### Install Python 3.6 Packages

In [ ]:
!pip install absl-py librosa soundfile matplotlib scipy scikit-learn numba jiwer unidecode deepspeech==0.8.2 praat-textgrids neptune-client

     |████████████████████████████████| 132 kB 37.7 MB/s 
     |████████████████████████████████| 203 kB 38.1 MB/s 
     |████████████████████████████████| 11.5 MB 236 kB/s 
     |████████████████████████████████| 22.2 MB 57 kB/s 
     |████████████████████████████████| 3.4 MB 54.9 MB/s 
     |████████████████████████████████| 241 kB 51.7 MB/s 
     |████████████████████████████████| 8.3 MB 61.0 MB/s 
     |████████████████████████████████| 303 kB 61.4 MB/s 
     |████████████████████████████████| 323 kB 64.2 MB/s 
     |████████████████████████████████| 377 kB 58.0 MB/s 
     |████████████████████████████████| 51 kB 535 kB/s 
     |████████████████████████████████| 25.3 MB 68 kB/s 
     |████████████████████████████████| 1.1 MB 41.5 MB/s 
     |████████████████████████████████| 50 kB 7.2 MB/s 
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23141 sha256=a49ff0092042a5540fa370a98200ce96285c6872667c6c30ac01b658acb8b66e
  Stored in directory: /root/.cache/pi

## Download and Check Datasets

### Download and Extract EEG Dataset from Google Drive

In [ ]:
# Download from Google Drive
!rm -rf emg_data
!EEG_DATA_ID="1GT_hZTCx0ihdjcPvQFgYltmNZU5iIxvj" && \
wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate "https://docs.google.com/uc?export=download&id=${EEG_DATA_ID}" -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=${EEG_DATA_ID}" -O emg_data.tar.gz && rm -rf /tmp/cookies.txt

--2021-07-19 00:18:09--  https://docs.google.com/uc?export=download&confirm=_NsN&id=1GT_hZTCx0ihdjcPvQFgYltmNZU5iIxvj
Resolving docs.google.com (docs.google.com)... 172.217.2.110, 2607:f8b0:4004:80a::200e
Connecting to docs.google.com (docs.google.com)|172.217.2.110|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-10-docs.googleusercontent.com/docs/securesc/38iasrdlsd1mkorkqn6urnch8q6o9dum/o2e9vu8m1m8th3eg4erj62p08siae62n/1626653850000/08446681616738227783/00659868113097021548Z/1GT_hZTCx0ihdjcPvQFgYltmNZU5iIxvj?e=download [following]
--2021-07-19 00:18:09--  https://doc-0o-10-docs.googleusercontent.com/docs/securesc/38iasrdlsd1mkorkqn6urnch8q6o9dum/o2e9vu8m1m8th3eg4erj62p08siae62n/1626653850000/08446681616738227783/00659868113097021548Z/1GT_hZTCx0ihdjcPvQFgYltmNZU5iIxvj?e=download
Resolving doc-0o-10-docs.googleusercontent.com (doc-0o-10-docs.googleusercontent.com)... 172.217.13.225, 2607:f8b0:4004:809::2001
Connecting to doc-0o-

In [ ]:
# Extract Dataset
!tar -xvf emg_data.tar.gz

Streaming output truncated to the last 5000 lines.
emg_data/nonparallel_data/5-2/278_audio.flac
emg_data/nonparallel_data/5-2/620_audio.flac
emg_data/nonparallel_data/5-2/717_emg.npy
emg_data/nonparallel_data/5-2/398_button.npy
emg_data/nonparallel_data/5-2/242_audio_clean.flac
emg_data/nonparallel_data/5-2/648_audio_clean.flac
emg_data/nonparallel_data/5-2/95_audio_clean.flac
emg_data/nonparallel_data/5-2/474_audio.flac
emg_data/nonparallel_data/5-2/32_button.npy
emg_data/nonparallel_data/5-2/564_button.npy
emg_data/nonparallel_data/5-2/266_info.json
emg_data/nonparallel_data/5-2/730_button.npy
emg_data/nonparallel_data/5-2/288_audio.flac
emg_data/nonparallel_data/5-2/508_info.json
emg_data/nonparallel_data/5-2/211_info.json
emg_data/nonparallel_data/5-2/472_button.npy
emg_data/nonparallel_data/5-2/455_emg.npy
emg_data/nonparallel_data/5-2/593_audio_clean.flac
emg_data/nonparallel_data/5-2/199_audio_clean.flac
emg_data/nonparallel_data/5-2/626_button.npy
emg_data/nonparallel_data/5-2/

### Download and Extract Phoneme Alignment Dataset from GitHub

In [ ]:
!rm text_alignments.tar.gz # Remove old one in case we're re-running cells
!wget -O text_alignments.tar.gz https://github.com/dgaddy/silent_speech_alignments/raw/main/text_alignments.tar.gz
!tar -xvf text_alignments.tar.gz

Streaming output truncated to the last 5000 lines.
text_alignments/5-8/5-8_33_audio.TextGrid
text_alignments/5-8/5-8_21_audio.TextGrid
text_alignments/5-8/5-8_206_audio.TextGrid
text_alignments/5-8/5-8_218_audio.TextGrid
text_alignments/5-8/5-8_152_audio.TextGrid
text_alignments/5-8/5-8_85_audio.TextGrid
text_alignments/5-8/5-8_62_audio.TextGrid
text_alignments/5-8/5-8_245_audio.TextGrid
text_alignments/5-8/5-8_111_audio.TextGrid
text_alignments/5-8/5-8_280_audio.TextGrid
text_alignments/5-8/5-8_170_audio.TextGrid
text_alignments/5-8/5-8_224_audio.TextGrid
text_alignments/5-8/5-8_197_audio.TextGrid
text_alignments/5-8/5-8_189_audio.TextGrid
text_alignments/5-8/5-8_133_audio.TextGrid
text_alignments/5-8/5-8_279_audio.TextGrid
text_alignments/5-8/5-8_108_audio.TextGrid
text_alignments/5-8/5-8_116_audio.TextGrid
text_alignments/5-8/5-8_242_audio.TextGrid
text_alignments/5-8/5-8_82_audio.TextGrid
text_alignments/5-8/5-8_155_audio.TextGrid
text_alignments/5-8/5-8_201_audio.TextGrid
text_ali

## GPU / TPU Check

In [ ]:
!nvidia-smi

import os
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime!')
else:
  print(os.environ["COLAB_TPU_ADDR"])

Mon Jul 19 00:22:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Download Silent Speech repo and Pre-Compiled WaveNet Library

### Download Silent Speech Repo

In [ ]:
!rm -rf silent_speech # In case we're re-running cells
!git clone https://github.com/MiscellaneousStuff/silent_speech/

Cloning into 'silent_speech'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 43 (delta 17), reused 32 (delta 9), pack-reused 0
Unpacking objects: 100% (43/43), done.


### Install Pre-Compiled WaveNet Library

#### Download Pre-Compiled WaveNet Library

In [ ]:
!cd /content/silent_speech && WAVENET_PRECOMPILED="1VHy97OiwpzlW5ta285syilrF0rgAh4Gp" && \
wget --no-check-certificate --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate "https://docs.google.com/uc?export=download&id=${WAVENET_PRECOMPILED}" -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=${WAVENET_PRECOMPILED}" -O nv_wavenet.zip && rm -rf /tmp/cookies.txt

--2021-07-19 00:22:21--  https://docs.google.com/uc?export=download&confirm=9SkZ&id=1VHy97OiwpzlW5ta285syilrF0rgAh4Gp
Resolving docs.google.com (docs.google.com)... 172.217.2.110, 2607:f8b0:4004:80a::200e
Connecting to docs.google.com (docs.google.com)|172.217.2.110|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-00-84-docs.googleusercontent.com/docs/securesc/d4cuef5sligj5vg6e735nec94l0hv60a/rd32kn34t8fnfa0ghboq7i3chkn1ue3h/1626654075000/08560850422083970408/11308906846717628304Z/1VHy97OiwpzlW5ta285syilrF0rgAh4Gp?e=download [following]
--2021-07-19 00:22:21--  https://doc-00-84-docs.googleusercontent.com/docs/securesc/d4cuef5sligj5vg6e735nec94l0hv60a/rd32kn34t8fnfa0ghboq7i3chkn1ue3h/1626654075000/08560850422083970408/11308906846717628304Z/1VHy97OiwpzlW5ta285syilrF0rgAh4Gp?e=download
Resolving doc-00-84-docs.googleusercontent.com (doc-00-84-docs.googleusercontent.com)... 172.217.13.225, 2607:f8b0:4004:809::2001
Connecting to doc-00-

#### Relocate Pre-Compiled Wavenet Library to Silent Speech repo

In [ ]:
!unzip /content/silent_speech/nv_wavenet.zip -d /content/silent_speech/

Archive:  /content/silent_speech/nv_wavenet.zip
   creating: /content/silent_speech/nv_wavenet/
 extracting: /content/silent_speech/nv_wavenet/pytorch.zip  
  inflating: /content/silent_speech/nv_wavenet/README.md  
  inflating: /content/silent_speech/nv_wavenet/nv_wavenet_util.cuh  
   creating: /content/silent_speech/nv_wavenet/.git/
  inflating: /content/silent_speech/nv_wavenet/.git/index  
   creating: /content/silent_speech/nv_wavenet/.git/modules/
   creating: /content/silent_speech/nv_wavenet/.git/modules/pytorch/
   creating: /content/silent_speech/nv_wavenet/.git/modules/pytorch/tacotron2/
  inflating: /content/silent_speech/nv_wavenet/.git/modules/pytorch/tacotron2/index  
   creating: /content/silent_speech/nv_wavenet/.git/modules/pytorch/tacotron2/branches/
  inflating: /content/silent_speech/nv_wavenet/.git/modules/pytorch/tacotron2/config  
   creating: /content/silent_speech/nv_wavenet/.git/modules/pytorch/tacotron2/info/
  inflating: /content/silent_speech/nv_wavenet/.

#### Install Pre-Compiled WaveNet Library and PyTorch Library

In [ ]:
!cd "./silent_speech/nv_wavenet/pytorch/" && python build.py install

running install
running bdist_egg
running egg_info
writing nv_wavenet_ext.egg-info/PKG-INFO
writing dependency_links to nv_wavenet_ext.egg-info/dependency_links.txt
writing top-level names to nv_wavenet_ext.egg-info/top_level.txt
reading manifest file 'nv_wavenet_ext.egg-info/SOURCES.txt'
writing manifest file 'nv_wavenet_ext.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
creating build/bdist.linux-x86_64/egg
copying build/lib.linux-x86_64-3.6/nv_wavenet_ext.cpython-36m-x86_64-linux-gnu.so -> build/bdist.linux-x86_64/egg
creating stub loader for nv_wavenet_ext.cpython-36m-x86_64-linux-gnu.so
byte-compiling build/bdist.linux-x86_64/egg/nv_wavenet_ext.py to nv_wavenet_ext.cpython-36.pyc
creating build/bdist.linux-x86_64/egg/EGG-INFO
copying nv_wavenet_ext.egg-info/PKG-INFO -> build/bdist.linux-x86_64/egg/EGG-INFO
copying nv_wavenet_ext.egg-info/SOURCES.txt -> build/bdist.linux-x86_64/egg/EGG-INFO
copying nv_wavenet_ext.

## Execute Desired Operation

In [ ]:
if OPERATION == "wavenet_model":
  !cd silent_speech && python wavenet_model.py --output_directory "/content/silent_speech/models/wavenet_model/" --silent_data_directories "/content/emg_data/silent_parallel_data" --voiced_data_directories "/content/emg_data/voiced_parallel_data,/content/emg_data/nonparallel_data"

elif operation == "transduction_model":
  !cd silent_speech && python transduction_model.py --pretrained_wavenet_model "/content/silent_speech/models/models/wavenet_model/wavenet_model.pt" --output_directory "/content/silent_speech/models/models/transduction_model/"

elif operation == "evaluate":
  !cd silent_speech && python evaluate.py --models "/content/silent_speech/models/transduction_model/model.pt" --pretrained_wavenet_model "/content/silent_speech/models/wavenet_model/wavenet_model.pt" --output_directory "/content/silent_speech/evaluation_output"

['wavenet_model.py', '--output_directory', '/content/silent_speech/models/wavenet_model/', '--silent_data_directories', '/content/emg_data/silent_parallel_data', '--voiced_data_directories', '/content/emg_data/voiced_parallel_data,/content/emg_data/nonparallel_data']
output example: (/content/emg_data/silent_parallel_data/5-6_silent, 208)
/content/silent_speech/nv_wavenet/pytorch/wavenet.py:46: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  self.conv.weight, gain=torch.nn.init.calculate_gain(w_init_gain))
/usr/local/lib/python3.6/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")